In [1]:
import os
import json
import torch
import faiss
import numpy as np
from PIL import Image, UnidentifiedImageError
from torchvision import transforms, models
from tqdm import tqdm

# =========================
# CONFIGURAZIONE
# =========================
gallery_folder = 'test/gallery'
query_folder = 'test/query'
top_k = 10

# Percorsi per caching
features_file = 'gallery_features.npy'
paths_file = 'gallery_paths.txt'

# Usa GPU se disponibile
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# =========================
# MODELLO VELOCE: ResNet18
# =========================
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Rimuove FC
model.to(device)
model.eval()

# =========================
# TRASFORMAZIONE
# =========================
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# =========================
# ESTRAZIONE FEATURE
# =========================
def extract_features(image_path):
    try:
        image = Image.open(image_path).convert('RGB')
    except UnidentifiedImageError:
        print(f"[ERRORE] Immagine non valida: {image_path}")
        return None
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        feature = model(image).squeeze().cpu().numpy()
    return feature

# =========================
# CARICA o CALCOLA GALLERIA
# =========================
if os.path.exists(features_file) and os.path.exists(paths_file):
    print("📂 Caricamento feature galleria salvate...")
    gallery_features = np.load(features_file)
    with open(paths_file, 'r') as f:
        valid_gallery_paths = [line.strip() for line in f.readlines()]
else:
    print("🧠 Estrazione feature dalla galleria...")
    gallery_paths = [os.path.join(gallery_folder, fname)
                     for fname in os.listdir(gallery_folder)
                     if fname.lower().endswith(('.jpg', '.jpeg', '.png'))]

    gallery_features = []
    valid_gallery_paths = []

    for p in tqdm(gallery_paths, desc="Estrai galleria"):
        feat = extract_features(p)
        if feat is not None:
            gallery_features.append(feat)
            valid_gallery_paths.append(p)

    gallery_features = np.array(gallery_features).astype('float32')
    np.save(features_file, gallery_features)
    with open(paths_file, 'w') as f:
        for path in valid_gallery_paths:
            f.write(path + '\n')

# =========================
# COSTRUISCI INDICE FAISS
# =========================
index = faiss.IndexFlatL2(gallery_features.shape[1])
index.add(gallery_features)

# =========================
# ELABORA QUERY
# =========================
results = []
query_paths = [os.path.join(query_folder, fname)
               for fname in os.listdir(query_folder)
               if fname.lower().endswith(('.jpg', '.jpeg', '.png'))]

top_k_effettivo = min(top_k, len(valid_gallery_paths))

print("🔍 Ricerca immagini simili...")
for q_path in tqdm(query_paths, desc="Query"):
    q_feat = extract_features(q_path)
    if q_feat is None:
        print(f"[ATTENZIONE] Query ignorata: {q_path}")
        continue

    q_feat = q_feat.astype('float32').reshape(1, -1)
    distances, indices = index.search(q_feat, top_k_effettivo)
    similar_images = [valid_gallery_paths[i] for i in indices[0]]
    results.append({
        "filename": q_path,
        "gallery_images": similar_images
    })

# =========================
# SALVA RISULTATI
# =========================
with open('submission_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✅ Ricerca completata. File salvato: submission_results.json")


📂 Caricamento feature galleria salvate...
🔍 Ricerca immagini simili...


Query:  11%|█         | 161/1455 [00:07<00:54, 23.60it/s]/home/disi/miniconda3/lib/python3.12/site-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Query:  95%|█████████▍| 1380/1455 [00:57<00:03, 24.17it/s]


KeyboardInterrupt: 

In [4]:
import json
import os

# Carica il file JSON
with open('submission_results.json', 'r') as f:  # Sostituisci con il tuo file
    data = json.load(f)

# Costruisci il dizionario nel formato desiderato
res = dict()
for entry in data:
    query_filename = os.path.basename(entry["filename"])
    gallery_filenames = [os.path.basename(p) for p in entry["gallery_images"]]
    res[query_filename] = gallery_filenames

# Esempio di stampa (prime 2 entry)
for k in list(res.keys())[:2]:
    print(f'{k}: {res[k]}')


3ccb9e24e6124427ad71e41f4a637593.jpg: ['272e1e5125944ea9a8513074c3047c89.jpg', '501fea8ef97940d0a569ad619189c40d.jpg', '0a3195260ab54d7f83c5ebff96b8a934.jpg', '1737f3015d3d4814bfe46aa6a6298a5e.jpg', '55987960e55541a983778be2751145e0.jpg', 'bc6a7f60881a44dda07f0e091cf8d288.jpg', '79fae434508140409c9898300c7b6413.jpg', '225e33d1f0764943a6e88f880da423cc.jpg', 'e97652e3b9ce4745ab96eb0a49f4bd30.jpg', '2883c9301c424c8ba0db796acabd0837.jpg']
abeb4b77d7694bf9ab67761d7f485269.jpg: ['0a400d67f9c24c5cb871e67b95af2280.jpg', '6e1fd42f80474defb740ff40638a119f.jpg', 'ed0976a829f54c6cba8edb541fefa051.jpg', '8b11436b42864cfcaba961db4e6dfcd9.jpg', '322fbafc147b4180b10b5af72fa4e353.jpg', '747a83fa3bf74542a947fcf4ae547eb3.jpg', 'b02514edcc0845e0a3cc528c56809e92.jpg', '36d4bce2b813451db9c33958d77b4182.jpg', '41c718023eeb42a6b11370c19c44f7e8.jpg', '48a439b60d9f4fedb2c2bdd1313cc395.jpg']


In [ ]:
# per visualizzare
import json
import matplotlib.pyplot as plt
from PIL import Image

# Load submission file
with open('submission_results.json', 'r') as f:
    results = json.load(f)

# How many queries to show?
num_queries_to_show = 5
top_k = len(results[0]['gallery_images'])

# Plot
for i, item in enumerate(results[:num_queries_to_show]):
    query_img = Image.open(item['filename']).convert('RGB')

    fig, axs = plt.subplots(1, top_k + 1, figsize=(3 * (top_k + 1), 4))
    axs[0].imshow(query_img)
    axs[0].set_title("Query")
    axs[0].axis('off')

    for j, gallery_path in enumerate(item['gallery_images']):
        gallery_img = Image.open(gallery_path).convert('RGB')
        axs[j + 1].imshow(gallery_img)
        axs[j + 1].set_title(f"Top {j+1}")
        axs[j + 1].axis('off')

    plt.tight_layout()
    plt.show()

In [5]:
import json
import requests


def submit(results, groupname, url="http://65.108.245.177:3001/retrieval/"):
    res = {}
    res["groupname"] = groupname
    res["images"] = results
    res = json.dumps(res)
    # print(res)
    response = requests.post(url, res)
    try:
        result = json.loads(response.text)
        print(f"accuracy is {result['accuracy']}")
    except json.JSONDecodeError:
        print(f"ERROR: {response.text}")

submit(res, "STF")


accuracy is 30.721649484536083


In [6]:
import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import random
import numpy as np
from sklearn.preprocessing import normalize

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dir = 'train'
gallery_dir = 'test/gallery'
query_dir = 'test/query'
batch_size = 64
epochs = 10
learning_rate = 1e-3
embedding_dim = 128
top_k = 10

# Augmentation for contrastive learning
simclr_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.8, 0.8, 0.8, 0.2),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# SimCLR Dataset
class SimCLRDataset(Dataset):
    def __init__(self, root_dir, transform):
        self.dataset = datasets.ImageFolder(root=root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, _ = self.dataset[idx]
        return self.transform(img), self.transform(img)

# Projection head
class ProjectionHead(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.ReLU(),
            nn.Linear(in_dim, out_dim)
        )

    def forward(self, x):
        return self.net(x)

# SimCLR model
class SimCLR(nn.Module):
    def __init__(self, base_model, projection_dim):
        super().__init__()
        self.backbone = nn.Sequential(*list(base_model.children())[:-1])
        self.projection_head = ProjectionHead(base_model.fc.in_features, projection_dim)

    def forward(self, x):
        x = self.backbone(x).squeeze()
        x = self.projection_head(x)
        return F.normalize(x, dim=-1)

# NT-Xent Loss
def nt_xent_loss(z1, z2, temperature=0.5):
    N = z1.size(0)
    z = torch.cat([z1, z2], dim=0)
    sim = F.cosine_similarity(z.unsqueeze(1), z.unsqueeze(0), dim=2)
    sim = sim / temperature

    labels = torch.arange(N, device=device)
    labels = torch.cat([labels + N, labels])

    mask = torch.eye(2 * N, dtype=torch.bool).to(device)
    sim = sim.masked_fill(mask, -9e15)

    targets = labels
    loss = F.cross_entropy(sim, targets)
    return loss

# Dataset and model
train_dataset = SimCLRDataset(train_dir, simclr_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

resnet = models.resnet18(pretrained=True)
model = SimCLR(resnet, embedding_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training
model.train()
for epoch in range(epochs):
    total_loss = 0
    for x1, x2 in train_loader:
        x1, x2 = x1.to(device), x2.to(device)
        z1 = model(x1)
        z2 = model(x2)
        loss = nt_xent_loss(z1, z2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# Save model
torch.save(model.state_dict(), "simclr_embedding_model.pth")

# Inference dataset
class ImageFolderPaths(Dataset):
    def __init__(self, paths, transform):
        self.paths = paths
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert("RGB")
        return self.transform(img), path

# Simple transform for test
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def get_image_paths(folder):
    return [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.jpg')]

def extract_embeddings(model, image_paths):
    dataset = ImageFolderPaths(image_paths, test_transform)
    loader = DataLoader(dataset, batch_size=batch_size)
    model.eval()
    features, paths = [], []
    with torch.no_grad():
        for images, pths in loader:
            images = images.to(device)
            emb = model(images).cpu().numpy()
            features.append(emb)
            paths.extend(pths)
    return np.vstack(features), paths

# Load model for inference
model = SimCLR(resnet, embedding_dim).to(device)
model.load_state_dict(torch.load("simclr_embedding_model.pth", map_location=device))
model.eval()

gallery_paths = get_image_paths(gallery_dir)
query_paths = get_image_paths(query_dir)

gallery_feats, gallery_paths = extract_embeddings(model, gallery_paths)
query_feats, query_paths = extract_embeddings(model, query_paths)

gallery_feats = normalize(gallery_feats, axis=1)
query_feats = normalize(query_feats, axis=1)

# Similarity + JSON output
results = []
for q_feat, q_path in zip(query_feats, query_paths):
    sims = gallery_feats @ q_feat
    topk_idx = np.argsort(-sims)[:top_k]
    topk_paths = [gallery_paths[i] for i in topk_idx]
    results.append({
        "filename": q_path,
        "gallery_images": topk_paths
    })

with open("submission1111.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ Saved submission_simclr.json")

/home/disi/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/disi/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [ ]:
import json
import os

# Carica il file JSON
with open('submission7.json', 'r') as f:  # Sostituisci con il tuo file
    data = json.load(f)

# Costruisci il dizionario nel formato desiderato
res = dict()
for entry in data:
    query_filename = os.path.basename(entry["filename"])
    gallery_filenames = [os.path.basename(p) for p in entry["gallery_images"]]
    res[query_filename] = gallery_filenames

# Esempio di stampa (prime 2 entry)
for k in list(res.keys())[:2]:
    print(f'{k}: {res[k]}')

In [ ]:
import json
import requests


def submit(results, groupname, url="http://65.108.245.177:3001/retrieval/"):
    res = {}
    res["groupname"] = groupname
    res["images"] = results
    res = json.dumps(res)
    # print(res)
    response = requests.post(url, res)
    try:
        result = json.loads(response.text)
        print(f"accuracy is {result['accuracy']}")
    except json.JSONDecodeError:
        print(f"ERROR: {response.text}")


submit(res, "STF")